In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from escnn.gspaces.r2 import rot2dOnR2
from escnn import nn as escnn_nn

In [42]:
# Hyperparameters
batch_size = 64
learning_rate = 0.001
epochs = 5

# Data transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load datasets
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [70]:
class EquivariantCNN(nn.Module):
    def __init__(self):
        super(EquivariantCNN, self).__init__()
        r2_act = rot2dOnR2(N=4)  # Rotational symmetry group

        # Define the layers
        self.input_type = escnn_nn.FieldType(r2_act, [r2_act.trivial_repr])
        self.conv1 = escnn_nn.R2Conv(
            escnn_nn.FieldType(r2_act, [r2_act.trivial_repr]), 
            escnn_nn.FieldType(r2_act, 8 * [r2_act.regular_repr]), 
            kernel_size=3, padding=1
        )
        self.bn1 = escnn_nn.InnerBatchNorm(escnn_nn.FieldType(r2_act, 8 * [r2_act.regular_repr]))
        self.relu1 = escnn_nn.ReLU(escnn_nn.FieldType(r2_act, 8 * [r2_act.regular_repr]))

        self.pool1 = escnn_nn.PointwiseAvgPoolAntialiased(
            escnn_nn.FieldType(r2_act, 8 * [r2_act.regular_repr]), 
            sigma=0.66, stride=2
        )

        self.conv2 = escnn_nn.R2Conv(
            escnn_nn.FieldType(r2_act, 8 * [r2_act.regular_repr]), 
            escnn_nn.FieldType(r2_act, 16 * [r2_act.regular_repr]), 
            kernel_size=3, padding=1
        )
        self.bn2 = escnn_nn.InnerBatchNorm(escnn_nn.FieldType(r2_act, 16 * [r2_act.regular_repr]))
        self.relu2 = escnn_nn.ReLU(escnn_nn.FieldType(r2_act, 16 * [r2_act.regular_repr]))

        self.pool2 = escnn_nn.PointwiseAvgPoolAntialiased(
            escnn_nn.FieldType(r2_act, 16 * [r2_act.regular_repr]), 
            sigma=0.66, stride=2
        )

        self.gpool = escnn_nn.GroupPooling(escnn_nn.FieldType(r2_act, 16 * [r2_act.regular_repr]))

        self.fc1 = nn.Linear(16 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = escnn_nn.GeometricTensor(x, self.input_type)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.gpool(x)
        x = x.tensor.view(x.tensor.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [71]:
# Initialize model, loss function, and optimizer
model = EquivariantCNN()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Testing loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')